In [2]:
import nltk
import numpy as np
import pandas as pd
import re
import string
import json
from nltk.corpus import reuters, stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.initializers import he_normal
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
# Download NLTK resources
nltk.download('reuters')
nltk.download('stopwords')

# Step 1: Tokenization and Preprocessing
class Tokenizer:
    def __init__(self, stop_words=None):
        self.stop_words = stop_words or set(stopwords.words('english'))
    
    def tokenize(self, text):
        text = text.lower()
        text = re.sub(f"[{re.escape(string.punctuation)}]", " ", text)
        text = re.sub(r"\d+", " ", text)
        tokens = text.split()
        return [word for word in tokens if word not in self.stop_words]

# Custom stopwords
custom_stopwords = set(stopwords.words("english"))
custom_stopwords.update({"mln", "lt", "pct", "dlrs", "vs", "000", "said", "year", "loss", "profit", "net"})

# Load Reuters dataset
documents = reuters.fileids()
texts = [reuters.raw(doc_id) for doc_id in documents]
labels = [reuters.categories(doc_id) for doc_id in documents]

tokenizer = Tokenizer(stop_words=custom_stopwords)
tokenized_texts = [tokenizer.tokenize(text) for text in texts]

# Step 2: Convert texts to Bag-of-Words representation
vectorizer = CountVectorizer(max_features=5000, stop_words=list(custom_stopwords))
X_bow = vectorizer.fit_transform([" ".join(tokens) for tokens in tokenized_texts])

# Step 3: LDA Topic Modeling
n_topics = 15
lda = LatentDirichletAllocation(n_components=n_topics, random_state=42)
X_lda = lda.fit_transform(X_bow)

# Display top words in each topic
print("LDA Topics:")
for idx, topic in enumerate(lda.components_):
    print(f"Topic #{idx + 1}:")
    print(" ".join([vectorizer.get_feature_names_out()[i] for i in topic.argsort()[-10:]]))

# Step 4: Load GloVe Embeddings
def load_glove_embeddings(glove_path, embedding_dim, vocabulary):
    embeddings_index = {}
    with open(glove_path, "r", encoding="utf-8") as file:
        for line in file:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype="float32")
            embeddings_index[word] = coefs
    
    embedding_matrix = np.zeros((len(vocabulary) + 1, embedding_dim))
    for word, idx in vocabulary.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[idx] = embedding_vector
    return embedding_matrix

glove_path = "glove.6B.100d.txt"  # Update path accordingly
embedding_dim = 100
vocabulary = {word: idx for idx, word in enumerate(vectorizer.get_feature_names_out())}
embedding_matrix = load_glove_embeddings(glove_path, embedding_dim, vocabulary)

# Step 5: Combine GloVe Embeddings with LDA Features
X_glove = X_bow.toarray()
X_combined = np.hstack((X_glove, X_lda))

# Step 6: One-Hot Encode Labels
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(labels)

# Step 7: Train/Test Split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)

# Step 8: Define Focal Loss Function
import tensorflow as tf

def focal_loss(gamma=2.0, alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        epsilon = tf.keras.backend.epsilon()
        y_pred = tf.keras.backend.clip(y_pred, epsilon, 1. - epsilon)
        pt = tf.where(tf.equal(y_true, 1), y_pred, 1 - y_pred)
        loss = -alpha * tf.math.pow(1. - pt, gamma) * tf.math.log(pt)
        return tf.reduce_mean(loss)
    return focal_loss_fixed

# Step 9: Build the Model
model = Sequential([
    Dense(512, activation="relu", input_shape=(X_train.shape[1],), kernel_initializer=he_normal()),
    BatchNormalization(),
    Dropout(0.3),
    Dense(256, activation="relu", kernel_initializer=he_normal()),
    BatchNormalization(),
    Dropout(0.3),
    Dense(y_train.shape[1], activation="sigmoid", kernel_initializer=he_normal())
])

# Compile the Model
model.compile(
    optimizer=Adam(learning_rate=0.0005),
    loss=focal_loss(gamma=2.0, alpha=0.25),
    metrics=["accuracy"]
)

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

# Train the Model
history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=64,
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

# Evaluate the Model
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=2)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

# Predict on Test Set
y_pred = model.predict(X_test)
y_pred_binary = (y_pred > 0.5).astype(int)

# Classification Report
print("\nClassification Report:")
print(classification_report(y_test, y_pred_binary, zero_division=0))

# Overall Accuracy
overall_accuracy = accuracy_score(y_test, y_pred_binary)
print(f"\nOverall Test Accuracy: {overall_accuracy}")

# Save the Model
model.save("glove_lda_dense_model.h5")
print("Model saved as 'glove_lda_dense_model.h5'")

[nltk_data] Downloading package reuters to
[nltk_data]     C:\Users\Colin\AppData\Roaming\nltk_data...
[nltk_data]   Package reuters is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Colin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
